# FUNZIONI DASK_CUDF CHE NON VANNO (PER FARE CONFRONTO)

## Questa è la sezione da portare per far vedere i limiti di dask_cudf

### Conversione colonna da stringa a datetime

Errore che restituisce:
"TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy()."

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
a = base.split('gas_start_date', 2)
a = a.rename(columns=cols)
a['month'] = a['month'].str.lower().map(mesi)
dd.to_datetime(a).head()

### Conversione colonna da stringa in formato "1,22 unita_misura" a float

In [ ]:
#Ottenere unità di misura
misure = base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]
#rimozione unità di misura
base.strip(['average_gas_bill_cost'], '€/smc')['average_gas_bill_cost']
#sostituzione della , con .
base.replace(['average_gas_bill_cost'], ',', '.', regex=True)['average_gas_bill_cost']

#### Le operazioni preliminari vanno a buon fine, qua si aprono più scenari (fallimentari)

questa soluzione è quella corretta su dask normale, utilizzando dask_cudf però genera un errore dovuto alla conversione implicita di un array in un numpy array, cosa non fattibile all'interno della GPU quindi consiglia di costruire esplicitamente una matrice tramite Cupy (Cuda Python) ma queste sono operazioni interne alla libreria

In [ ]:
dd.to_numeric(base.get_df()['average_gas_bill_cost'])
 

Qua da 2 diverse tipologie di errori: una dovuta ai metadati e l'altra alla presenza di valori non convertibili, anche togliedo tutti i valori nan con [~base['average_gas_bill_cost'].isna()] la situazione non cambia

In [ ]:
base.get_df()['average_gas_bill_cost'].map_partitions(cudf.to_numeric, 'raise')

# Pipeline

In [ ]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

In [ ]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

In [ ]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

# Loading Dataset

In [1]:
import dask_cudf as dc
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cudf
import numpy as np
#import graphviz
from base_dask_cudf import BaseDfBench
base = BaseDfBench("DASK")
# base = BaseDfBench()

In [2]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

# Uncomment the line under this one to load the pre-converted dataset
base.load_dataset('parquet', path='data/mod_parquet')
# base.load_dataset('parquet')
df = base.get_df()
columns = base.get_columns()
base.npartitions()

In [3]:
# Dataset's rows
# I commented the real computation because it takes a lot of time
# dataset_rows = base.get_df().shape[0].compute()
dataset_rows = 10497143
dataset_rows

In [ ]:
#verica di user_code+customer_code = PK utility
#PK customer = user_code
#PK invoice = bill_id
df.groupby(by=['user_code', 'customer_code'])['bill_id'].count().compute()

## Load big Dataset (DEPRECATED)

In [ ]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

In [ ]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(f"DF partitions: {ddf.npartitions}")
print(f"DF: {ddf}")

# UTILS

In [4]:
# CONVERTING ALL THE STRING COLUMNS TO OBJECT COLUMN
# THIS BECAUSE STRING COLUMNS CAUSE A LOT OF ERRORS WHEN PROCESSED AS STRINGS
for k,v in base.get_columns_types().items():
    if v == 'string':
        base[k] = base[k].astype('object')
    elif v == 'Float64':
        base[k] = base[k].astype('float')

In [5]:
# Seguendo il GroundTruth mystico
mappone = {'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'datetime64[ns]',
 'light_start_date': 'datetime64[ns]',
 'light_end_date': 'datetime64[ns]',
 'tv': 'float64',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[ns]',
 'supply_type': 'object',
 'gas_start_date': 'datetime64[ns]',
 'gas_end_date': 'datetime64[ns]',
 'extra_fees': 'float64',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'float64',
 'light_offer_type': 'object',
 'light_offer': 'object',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'object',
 'billing_frequency': 'object',
 'bill_type': 'object',
 'gas_system_charges': 'float64',
 'light_system_charges': 'float64',
 'gas_material_cost': 'float64',
 'light_transport_cost': 'float64',
 'gas_transport_cost': 'float64',
 'light_material_cost': 'float64'
}

In [6]:
#Check sui tipi di dato attuali
type_dict = base.get_columns_types()
for k,v in type_dict.items():
    if v != mappone[k]:
        print(f'mismatch in {k} with values: {v}, {mappone[k]}')

In [7]:
# Trovare le tipologie di colonne a seconda della tipologia
int_cols = base.col_type(find=['int'])
string_cols = base.col_type(find=['string']) #object
float_cols = base.col_type(find=['float'])
numeric_cols = base.col_type(find=['numeric'])
date_cols = base.col_type(find=['date'])

# Convert Columns

## Da portare

In [ ]:
# DONE gas_amount, howmuch_pay, total_amount, light_amount to float
# DONE gas_average_col, gas_offer, average_unit_light_cost, average_light_bill_cost, average_unit_gas_cost, average_gas_bill_cost deve essere float non int, basta un astype()
# light_offer è segnato come string perché dovrebbe essere un nome ma nel dataset è un int (what)
# il caso light_offer risolto: in teoria è str ma in pratica è stato anonimizzato, provo il cast to string

In [ ]:
base['light_offer'] = base['light_offer'].astype(str)

In [ ]:
len(base.col_type('string'))

### Date conversion

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
#finding columns with date keyword
date_cols = []
for c in df.columns:
    # print(base[c].dtype)
    if 'date' in c.split('_') and base[c].dtype != 'datetime64[ns]':
        date_cols.append(c)

print(f'columns to convert to date: {date_cols}')
#converting these columns
for c in date_cols:

    print(c , df[c].dtype)
    
    a = base.split(c, 2)
    a = a.rename(columns=cols)
    a['month'] = a['month'].str.lower().map(mesi)
    base.get_df()[c] = dd.to_datetime(a, format="%d/%m/%Y")
    # print(a.head())
    
    #df[c] = base.split(c, 2)
    
    # print(base[c].head())

In [ ]:
base.df

### Removing measure unit from strings and converting them into numbers

In [ ]:
# this line can be used to retrieve alle the columns with a number pattern inside them but is too resources consuming
# Due to the small number of columns We preferred to analize them one by one

# base['howmuch_pay'].str.contains('[0-9]+[.,][0-9]*', regex=True).count().compute()
# [0-9]+[.,][0-9]*

In [ ]:
#finding string columns with number inside
to_numeric_cols = []
for c in df.columns:
    if ('average'  in c.split('_') or 'amount' in c.split('_')) and (base[c].dtype in ['object', 'str']):
        # print(base[c].dtype)
        to_numeric_cols.append(c)

# Added manually after column analysis 
to_numeric_cols.append('howmuch_pay')
to_numeric_cols.append('tv')
to_numeric_cols.append('extra_fees')
print(to_numeric_cols)

In [ ]:
# Exctracting measure unit from the column
# Commented bc is time consuming
# base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]

In [ ]:
# Converting columns from string tu numeric
partitions = base.npartitions()
for c in to_numeric_cols:
    base[c] = base[c].str.extract(r'([0-9]+[,.][0-9]*)', flags=0, expand=False)
    # print(f'{c} measure: {measure}')
    base.replace([c], ',', '.', regex=True)
    base[c] = dd.to_numeric(base[c], errors='coerce', meta=('float'))
# base.get_df().head()['average_gas_bill_cost']

In [ ]:
base.df

In [ ]:
# Strip all the columns in order to remove all the whitespaces 
base.strip(string_cols)

In [ ]:
base.df = base.df.persist()

In [ ]:
base['howmuch_pay'].head()

### Save the modified dataset

In [ ]:
# Save the converted dataset
base.to_parquet('data/mod_parquet')

In [ ]:
base.load_dataset('parquet', path='data/mod_parquet')
base.get_columns_types()

## Da non portare

In [ ]:
# Dovrebbe accorgersi di colonne con similarità > soglia
# TODO --> FUNZIONE DA RIFARE PERCHé USA I SET E QUINDI NON CONFRONTA RIGA-RIGA 
#          MA SET(ELEMENTI_COL1) CON SET(eLEMENTI_COL2)
def get_duplicate_col(df, soglia=0.9):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """  
        cols = df.columns.values
        #return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
        
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if len(set(cols[i]) & set(cols[j])) / float(len(set(cols[i]) | set(cols[j]))) > soglia ]

# 1. INGESTION & DISCOVERY

## Analysis DA PORTARE

### Columns gas related

gas_amount, gas_average_cost, gas_start_date, gas_end_date, gas_consumption, gas_offer, average_unit_gas_cost, average_gas_bill_cost, gas_system_charges, gas_material_cost, gas_transport_cost

In [46]:
base[['gas_amount', 'gas_average_cost', 'gas_start_date', 'gas_end_date', 'gas_consumption', 'gas_offer', 'average_unit_gas_cost', 'average_gas_bill_cost', 'gas_system_charges', 'gas_material_cost', 'gas_transport_cost']].tail()
# 'gas_average_cost' and 'average_unit_gas_cost' are equivalent

,gas_amount,gas_average_cost,gas_start_date,gas_end_date,gas_consumption,gas_offer,average_unit_gas_cost,average_gas_bill_cost,gas_system_charges,gas_material_cost,gas_transport_cost
461174,NaN,NaN,2021-05-27,2021-04-03,NaN,18446744073709551615,NaN,NaN,NaN,NaN,NaN
469346,NaN,NaN,2021-05-27,2021-03-27,NaN,18446744073709551615,NaN,NaN,NaN,NaN,NaN
399408,NaN,NaN,2021-05-27,2021-03-24,NaN,18446744073709551615,NaN,NaN,NaN,NaN,NaN
275461,0.0,NaN,2021-04-07,2019-10-01,0.0,2533091950655969294,NaN,NaN,0.0,0.0,0.0
402530,NaN,NaN,2021-05-31,2020-08-01,NaN,18446744073709551615,NaN,NaN,NaN,NaN,NaN


### Columns light related

F1_kWh, F2_kWh, F3_kWh, light_start_date, light_end_date, light_average_cost, light_consumption, light_offer_type, light_offer, light_amount, average_unit_light_cost, average_light_bill_cost, light_system_charges, light_transport_cost, light_material_cost

Riflessioni: 
light_consumption * average_light_bill_cost = light_amount, vero ma c'è un piccolissimo scarto che non li fa combaciare.

light_consumption * average_unit_light_cost = light_material_cost, soldi spesi dall'azienda per comprare l'energia anche qua c'è un piccolissimo scarto

light_amount - light_material_cost = guadagno dell'azienda, a cui bisogna togliere i costi di trasporto, light_system_charges

In [53]:
base[['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_start_date', 'light_end_date', 'light_average_cost', 'light_consumption', 'light_offer_type', 'light_offer', 'light_amount', 'average_unit_light_cost', 'average_light_bill_cost', 'light_system_charges', 'light_transport_cost', 'light_material_cost']].sample(frac=0.001).compute()

,F1_kWh,F2_kWh,F3_kWh,light_start_date,light_end_date,light_average_cost,light_consumption,light_offer_type,light_offer,light_amount,average_unit_light_cost,average_light_bill_cost,light_system_charges,light_transport_cost,light_material_cost
264150,-33.0,105.0,107.0,2021-03-27,2021-05-27,28.0,179.0,light single zone,9419031583198849335,52.28,0.08,0.29,7.48,20.93,15.04
114404,-2.0,-1.0,-1.0,2021-02-20,2021-03-19,NaN,-4.0,light single zone,5384502039674139572,16.81,NaN,NaN,-0.16,6.99,8.46
267865,-20.0,-23.0,-40.0,2020-08-01,2021-02-22,NaN,-83.0,light multi zones,13551778016618737280,219.64,NaN,NaN,72.60,75.53,52.59
410415,-2.0,-1.0,-1.0,2020-11-28,2021-01-26,NaN,-4.0,light single zone,15455868716410888739,45.99,NaN,NaN,19.71,13.91,8.28
158694,-9.0,-8.0,-6.0,2021-04-03,2021-05-31,NaN,-23.0,light single zone,11439497290139626097,47.96,NaN,NaN,19.66,13.84,10.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38935,298.0,245.0,344.0,2020-12-05,2021-02-02,37.0,887.0,light single zone,15455868716410888739,164.59,0.07,0.19,37.08,28.57,64.07
432592,2055.0,1436.0,1666.0,2021-01-27,2021-03-26,40.0,3236.0,light multi zones,11219961771986969242,956.14,0.12,0.30,248.96,113.10,381.21
10022,310.0,188.0,238.0,2021-03-20,2021-05-21,44.0,736.0,light single zone,15455868716410888739,132.02,0.08,0.18,30.73,20.57,55.46
379962,546.0,408.0,379.0,2020-12-19,2021-01-22,48.0,1333.0,light single zone,556925197644088568,307.83,0.11,0.23,55.74,65.17,147.05


F1_kWh + F2_kWh + F3_kWh = light_consumption? 

In [58]:
base[(base['light_consumption'] == (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))].shape[0].compute()

4652400

In [61]:
F_kWh = base[['F1_kWh', 'F2_kWh', 'F3_kWh']]
F_kWh = F_kWh.persist()

In [67]:
# row with all the values not null
F_kWh[~F_kWh.isna().any(axis=1)].shape[0].compute()

10496446

In [69]:
# row with all values null
F_kWh[F_kWh.isna().all(axis=1)].shape[0].compute()

85

In [70]:
# rows with at least 1 value null
F_kWh[F_kWh.isna().any(axis=1)].shape[0].compute()

697

In [71]:
F_kWh['sum'] = F_kWh.sum(axis=1)

In [78]:
F_kWh[F_kWh[['F1_kWh', 'F2_kWh', 'F3_kWh']].isna().all(axis=1)].compute()

,F1_kWh,F2_kWh,F3_kWh,sum
475464,NaN,NaN,NaN,0.0
519650,NaN,NaN,NaN,0.0
115745,NaN,NaN,NaN,0.0
258881,NaN,NaN,NaN,0.0
302809,NaN,NaN,NaN,0.0
...,...,...,...,...
180094,NaN,NaN,NaN,0.0
220513,NaN,NaN,NaN,0.0
239335,NaN,NaN,NaN,0.0
275461,NaN,NaN,NaN,0.0


In [68]:
F_kWh.shape[0].compute()

10497143

### All the record with city == VIETRI DI POTENZA have address and nominative null and all the addresses and nominative null values are the VIETRI DI POTENZA records

In [12]:
#esempio
base.locate_null_values('address').shape[0].compute()

In [14]:
base[base['city'] == 'VIETRI DI POTENZA'].shape[0].compute()

In [16]:
base[base['nominative'].isna()].shape[0].compute()

## Locate Missing Values DA PORTARE

In [29]:
# LOCATE MISSING VALUES
#rows = df.shape[0]
#wait(ddf.shape[0])
#print("Total rows", rows)

# These are the colums with at least one missing value
column_count_nan = {}
# compute the presence of null value on time instead every iteration for performance reasons
na = base.get_df().isna()
val = na.any().compute()

res = na.sum().compute()
column_count_nan = json.loads(res.to_json())
column_count_nan

In [30]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
with open('data/initial_null_val.json', 'w+') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(column_count_nan, f, cls=NpEncoder, indent=2)

In [31]:
import json 

with open('data/initial_null_val.json', "r") as read_file:
    column_count_nan = json.load(read_file)

In [34]:
column_with_nan = []

perc = 80
soglia = ((perc * base.get_df().shape[0]) / 100).compute()
print(f'soglia {soglia}')

for c, val in column_count_nan.items():
    #print(f'{c}')
    if val > soglia:
        print(f'{c} val: {val}')
        column_with_nan.append(c)
        
column_with_nan

# Si può notare come nessuna colonna abbia il 100% di valori nulli, ma ci sono altresì tante 
# colonne con un numero elevato di nan. Poche colonne hanno un numero basso di nan.

### Analisi extra (utile estenderla all'intero dataset)

In [ ]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative').shape[0].compute()

In [ ]:
base.locate_null_values('address').shape[0].compute()

In [35]:
df[df['city'] == 'VIETRI DI POTENZA'].shape[0].compute()

Not all the user code are repeated, but even in the repeated we can't be sure if the 2 records correspond, this because the real name are hased and possible collission can exists

In [41]:
df['user_code'].value_counts().shape[0].compute()

In [37]:
# PROVO A VEDERE SE SONO PRESENTI ALTRI RECORDS DEGLI UTENTI A CUI MANCANO NOMINATIVE E ADDRESS
v = df['user_code'].value_counts()
v[v.gt(1)].index.compute()

In [40]:
# DATASET CON GLI user_code CHE COMPAIONO PIù DI UNA VOLTA
df[df['user_code'].isin(v[v.gt(1)].index.compute())].compute()

In [ ]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

#NOTA PER DASK, SE FACCIAMO 0 CI TORNA IL PRIMO RECORD DI TUTTE LE 52 PARTIZIONI
df.loc[0, 'billing_frequency'].compute()

## Locate Outliers DA PORTARE 

In [42]:
base['F1_kWh'].quantile(0.1).compute()

In [43]:
#CERTE COLONNE CON IL TIPO DI DATO PARTICOLARE TIPO FLOATDTYPE64 VANNO CONVERTITE IN TIPI COMPRENSIBILI
base.locate_outliers('F1_kWh').head()

In [ ]:
type = df.dtypes
num = type[type!='object'][type!='bool']
num = num.index.to_list()
num

In [ ]:
tmp = ddf.locate_outliers("F1_kWh")
tmp = tmp.compute()
tmp.shape

In [ ]:
for col in num:
    tmp = base.locate_outliers(col)
    tmp = tmp.compute()
    
    print(f"{col} has {tmp.shape[0]} outliers")

## Sort Data DA PORTARE

In [44]:
base.sort("F1_kWh").head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
489056,1187360,-161851.0,-72788.0,-32862.0,ACQUAVIVA PICENA,5CE681F589E5A3772769595584C0EB928B7DE066,3FBEB23E4339852AF45CB861CC4C6E9ADD80F327,P,0,E63583F6784908CB23E33CE0EFEE5643903B02D2,...,NaN,795C9C69BB38265B7A23E6E97B8EA379111419DD,None,False,NaN,221.90,NaN,106.01,NaN,198.47
65588,1044765,-155575.0,-71565.0,-167583.0,BOTRUGNO,E118655788098B92955F2B187B5E635315F40418,7AF193111F1FE1134095ACFB7E55FB28B1D70CA5,P,51,9891E03957AE0D2613A3D9AA07883B57965D3F85,...,NaN,796F4BE2B2D0B882778FFB1F5800742493915391,None,False,NaN,1252.72,NaN,327.57,NaN,1728.48
356661,2449406,-155575.0,-71565.0,-167583.0,BOTRUGNO,E118655788098B92955F2B187B5E635315F40418,7AF193111F1FE1134095ACFB7E55FB28B1D70CA5,P,51,9891E03957AE0D2613A3D9AA07883B57965D3F85,...,NaN,796F4BE2B2D0B882778FFB1F5800742493915391,None,False,NaN,0.39,NaN,0.07,NaN,-0.68
340207,1319384,-153991.0,-98086.0,-231472.0,SAN GIOVANNI LA PUNTA,01E389112E8D04323E5C88E5DAD63B50A137AEF6,874BAF9F76FF33F516411DCF0BF3C484210EEB2E,P,0,1CFC1FF48D1B1272895365B3BD01E89A13AE049B,...,NaN,F57EC4016CFA6F1AC121561E1586BDC3D6EE5541,monthly,False,NaN,147.89,NaN,62.17,NaN,229.25
19920,1434857,-153137.0,-123193.0,-195310.0,ACQUANEGRA CREMONESE,A4D4CD0A605BEAA3822EF0A59A1AD76A7CA57CF8,D2008C12D5E4C96C1ECDD4B6E1B01397FD939072,M,74,6132481E9E67205B05D91820BBCF68D6A4F3DBC0,...,1.84,30709EDF7F80B0875A78FA53721A0AA60262B91E,None,False,-3.94,1006.22,18.48,300.64,13.39,1723.67


### Bazze mirco

In [ ]:
tmp = ddf.sort_index("F1_kWh").compute()
# not working

In [ ]:
idx = tmp.index.to_arrow().to_pylist()
# too mutch long list

In [ ]:
tmp.merge(ddf.df, on=["F1_kWh"], how="left")

# 2. VALIDATION

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

## Check Data Range DA PORTARE TUTTE (mi tirava fare un sottogruppo)

### Age nulla per PIVA

In [ ]:
# Controllo se chi ha P ha età nulla
import pandas as pd
df['age'] = df['age'].mask((df['sex']=='P') & (df['age']>=0), np.nan)
df[(df['sex']=='P')][['sex','age']].sample(frac=0.1).compute()

### Age >= 18

In [ ]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
# Dovrebbe corrispondere al numero di PIVA
df['age'].isna().sum().compute()

In [ ]:
# Che fare con i minorenni? Li mettiamo tutti a 18 anni?
df[(df['age'] < 18)].shape[0].compute()

### Translation in English

In [ ]:
df['supply_type'].value_counts(dropna=False).compute()

In [ ]:
# Dobbiamo metterli in inglese? Si può fare con un map
ing = {
    'luce': 'light',
    'gas e luce': 'gas and light',
    'gas' : 'gas',
}
df['supply_type'] = df['supply_type'].map(ing)

In [ ]:
df['supply_type'].value_counts(dropna=False).compute()

### Data control

### billing_freq & bill_type Analysis

In [ ]:
# Che fare qua? Billing frequency (‘monthly’, ‘quarterly’, etc.)
# Si potrebbero calcolare
df['billing_frequency'].value_counts(dropna=False).compute()

In [ ]:
# Kind of invoice (False means a “standard bill”)
df['bill_type'].value_counts(dropna=False).compute()

### light_offer_type Analysis

In [ ]:
# Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
df['light_offer_type'].value_counts(dropna=False).compute()

In [ ]:
# I 2 valori di light mi puzzano, famo uno strip?
df['light_offer_type'] = df['light_offer_type'].str.strip()
df['light_offer_type'].value_counts(dropna=False).compute()
# GHAVEVO RAGIONE Vè

## Check Column Uniqueness DA PORTARE E DA VERIFICARE

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [ ]:
## Sserve il date formatter!!
df['emission_date'].head()

## Find Data Mismatch

In [ ]:
# VERY time consuming, expecially with DASK, with DASK_CUDF It is a lot faster but It takes a lot of time as well
# base.get_duplicate_columns()
# Ne vengono individuate solo 5 di coppie 100% fedeli, 
# ma cambia la questione se abbassiamo la soglia? Ci sono degli errori sopra perchè 
# quelle 100% uguali non vengono riconosciute, mentre qua sì
duplicates = [('date', 'light_start_date'),('date', 'gas_end_date'),('light_start_date', 'gas_end_date'),('light_end_date', 'gas_start_date'),('gas_average_cost', 'average_unit_gas_cost')]
duplicates

In [ ]:
df['date'] == df['light_start_date']

In [ ]:
base['tv'].head()

# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #

In [ ]:
#poichè probabilmente con i <NA> ho un sacco di problemi "lasciatemi castaareeee.. 
#con la chitarra in maaanoooo, lasciatemi castare.. sono un italiano"

df = base.fill_nan(np.nan)
df

In [ ]:
# setto le colonne con i tipi di dato in mappone
df = base.cast_columns_types(mappone)

In [ ]:
df['emission_date']

In [ ]:
# Abbiamo dei valori non castabili a float nelle colonne interessate
# Vediamo cos'ho nelle colonne str
#print(string_cols)
for c in string_cols:
    print("Col: ", c)
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(cosa)

In [ ]:
# C'è della merda, ottimizzo la cosa riducendo il campo d'analisi a solo le colonne float64 di mappone
mappone_float64_cols = []

for (k, v) in mappone.items():
    if v == 'float64':
        mappone_float64_cols.append(k)

final_float_cols = [x for x in mappone_float64_cols if x not in float_cols]
mappone_float64_cols
float_cols
final_float_cols = [x for x in final_float_cols if x not in ['gas_offer']]

In [ ]:
# Altro caso di colonna dal dubbio gusto
df['gas_offer']

In [ ]:
# Vediamo cos'ho nelle colonne str
#print(string_cols)
lista_rename = []
for c in final_float_cols:
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    if cosa != '0.00' and cosa != 'nan':
        print("Col: ", c + ' ' + cosa)
        print(cosa)
        p = {
            c : c + ' ' + cosa
        }
        lista_rename.append(p)
        df[c] = df[c].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)').astype(float)
df

In [ ]:
# voglio trovare le colonne str che diventeranno float che generano problemi
# VIRGOLE DI MERDA
problematiche = [x for x in string_cols if x in final_float_cols]
for p in problematiche:
    print(p)
    df[p] = df[p].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(float(df[p][0]))

In [ ]:
float(0.48)

In [ ]:
# Ho diverse colonne da cui eliminare le unità di misura...
df['gas_average_cost'].dtype

In [ ]:
# in 'bill_type' False == 'standard bill'
# Funziona solo se la funzione sopra non va in errore.

df = base.replace('bill_type', 'False', 'standard bill', False)
df

In [ ]:
#Verifica della bontà della sostituzione
df['bill_type'].value_counts()

In [ ]:
# Non c'è nessuna riga tutta nulla, Giulio è contento
df.dropna(how='all')

# 3. STRUCTURING

In [ ]:
''' 
+++++ DATA STRUCTURING +++++
    • Change column data types
    • Delete, split or merge columns
    • Pivot and unpivot
'''

## Change Column Data Types 

In [ ]:
# aggregabile con lo step precedente
# L'HO FATTO SOPRA, è DA ORDINARE E PERFEZIONARE

## Delete, Split or Merge Columns

In [ ]:
# rimozione colonne duplicate 

## Pivot and Unpivot

In [ ]:
# non saprei cosa farci

# 4. ENRICHMENT

In [ ]:
''' 
+++++ DATA ENRICHMENT +++++
    • Calculate columns using expressions
    • Set primary key column
    • Join or append DataFrames
    • Group by and aggregate records
    • Scale column values into a certain range
    • Encode categorical data (one-hot encoding and label encoding)
'''

## Calculate Columns Using Expressions

## Set PK column

# 5. FILTERING

In [ ]:
''' 
+++++ DATA FILTERING +++++
    • Sample rows
    • Select a subset of rows through a user-defined query
'''

## Sample Rows

In [ ]:
df.sample(20)

In [ ]:
# SELECT A SUBSET OF ROWS?!?

# 6. CLEANING

In [ ]:
''' 
+++++ DATA CLEANING +++++
    • Change data format and case
    • Deduplicate data
    • Missing value imputation
    • Find and replace values
'''

In [ ]:
# CHANGE DATA FORMAT AND CASE
# data format dovrebbe già essere fatto a questo punto
# case pure se vogliamo aver fatto tutto per bene

In [ ]:
# DEDUPLICATE DATA
# controllo se ho più bollette uguali dello stesso cliente

In [ ]:
ddf

In [ ]:
# MISSING VALUE IMPUTATION
# qua? Spero di averne pochissimi nel caso

In [ ]:
# FIND AND REPLACE VALUES
# ossia?

In [ ]:
x = np.nan
float(x)